N-grams 
----------------------

## French Part

Les n grams sont utilisés dans le but de connaître le contexte d'un mot ou d'un groupe de mots dans un texte. Autrement dit, considérons une intelligence artificielle (ab. IA) qui souhaite connaître le sens de chaque mot dans un texte dans le but de prédire le sentiment que l'auteur a voulu exprimer. l'IA ne sera pas capable d'effectuer cette tâche sans une certaine connaîssance de la grammaire formelle. Cela est impossible à cause du nombre important de règles à adopter. C'est là que les n-grams vont entrer en jeu. Aulieu d'apprendre les règles de la grammaire francaise, l'IA va devoir essayer de reconnaitre des parties du texte qu'on lui fourni et non la globalité de ce dernier.

**Définition du n-gram d'un texte**: Le n-gram est un ensemble quelconque de n éléments contigues d'un texte. Cet élément peut-être :
- Un mot : On divise le texte en un ensemble de parties constituées de n mots par pas de 1 mot.
- Un caractère: On divise le texte en un ensemble de parties constituées de n caractères par pas de 1 caractère. 

Les éléments récupérés peuvent contenir ou pas de ponctuation(s) suivant la tâche que l'on souhaite accomplir.

**Définition de la tokénisation**: La tokénisation est la séparation d'un texte en un ensemble d'éléments unitaires. Il peut s'agîr :
- De la séparation du texte en mots,
- Ou en phrases séparées par des points. On peut effectuer la séparation par phrase pour ensuite effectuer la séparation par mot pour éviter les ponctuations ou pour analyser chaque phrase séparément.

Les éléments récupérés après tokénisation d'un texte (*tokens*) peuvent servir pour la constitution des n-grams.

**Les étiquettes**: Les étiquettes, en anglais pos-tag, représentent la grammaire hors contexte d'un texte à partir de ces tokens. Elle nous permettront notamment de choisir les n-grams ne contenant pas de ponctuation.

**Exemple simple**: Effectuons un exemple simple avec `nltk`.

Importons la librairie

In [1]:
import nltk

Initialisons un texte constituées de deux phrases

In [8]:
texte = "Les papiers sont stériles; L'audace et la romance semblent être passées à jamais dans le monde criminel. Pouvez-vous alors me demander si je suis prêt à examiner un problème nouveau, aussi insignifiant soit-il?"

Passons à la tokénisation du texte (en phrases puis en mots) sans considérer la ponctuation

Vu que nous utilisons un texte en francais, nous allons donc ajouter l'argument `language = "french"`

In [3]:
# phrases
phrases = nltk.sent_tokenize(texte, language="french") 

In [23]:
# mots
mots = []
ponctuations = ".,?!;"

# utilisation de tokéniseur utilisant des expressions régulières
tokenizer = nltk.RegexpTokenizer(r"[\w']+") # récupère uniquement les mots

for i, phrase in enumerate(phrases):
    mots.append(tokenizer.tokenize(phrase))

In [24]:
mots

[['Les',
  'papiers',
  'sont',
  'stériles',
  "L'audace",
  'et',
  'la',
  'romance',
  'semblent',
  'être',
  'passées',
  'à',
  'jamais',
  'dans',
  'le',
  'monde',
  'criminel'],
 ['Pouvez',
  'vous',
  'alors',
  'me',
  'demander',
  'si',
  'je',
  'suis',
  'prêt',
  'à',
  'examiner',
  'un',
  'problème',
  'nouveau',
  'aussi',
  'insignifiant',
  'soit',
  'il']]

Nous obtenons ci-dessus une liste de liste de mots pour une phrase du texte donné.

Voici une fonction qui récupère les n-grams

In [18]:
def get_n_grams(tokens: list, n: int = 2):
    n_grams = []
    for i in range(len(tokens) - n):
        n_grams.append(" ".join(tokens[i: i+n])) 
    return n_grams       

Nous allons pouvoir passer à l'affichage des n-grams:

- D'abord pour la première phrase par pas de 2

In [34]:
get_n_grams(mots[0], 2)

['Les papiers',
 'papiers sont',
 'sont stériles',
 "stériles L'audace",
 "L'audace et",
 'et la',
 'la romance',
 'romance semblent',
 'semblent être',
 'être passées',
 'passées à',
 'à jamais',
 'jamais dans',
 'dans le',
 'le monde']

- Ensuite pour la deuxième phrase par pas de 3

In [35]:
get_n_grams(mots[1], 3)


['Pouvez vous alors',
 'vous alors me',
 'alors me demander',
 'me demander si',
 'demander si je',
 'si je suis',
 'je suis prêt',
 'suis prêt à',
 'prêt à examiner',
 'à examiner un',
 'examiner un problème',
 'un problème nouveau',
 'problème nouveau aussi',
 'nouveau aussi insignifiant',
 'aussi insignifiant soit']

#### Markov assumption (hypothèse de Markov)

Après l'obtention des n-grams il nous est possible de prédire le prochain mot dans une séquence à partir des précédents mots la composant. Pour cela on doit calculer, pour chaque possible prochain mot, la probabilité que celui occure sachant que les mots précédents font déja partis de la séquence.

Exemple :

Pour calculer la probabilité que le prochain mot de la sous séquence "L'audace et la romance semblent être passées à jamais dans le monde" soit le mot "criminel" nous aurons:

$$
P(prochain\_mot = \text{"criminel"} / \text{sous-séquence}) =
$$
$$
\frac{\text{nombre d'occurences "L'audace et la romance semblent être passées à jamais dans le monde criminel"}}{\text{nombre d'occurences "L'audace et la romance semblent être passées à jamais dans le monde"}}
$$

Cette méthode est peut recommendable car elle peut biaiser notre résultat car dans la plupart des cas le nombre d'occurences d'une sous-séquence aussi grande est très faible (surtout dans notre cas où les phrases ont une composition très complexe).

Pour pallier à ce problème nous allons à la place utiliser les n-grams pour ne prendre comme sous-séquence que les n mots précédents le possible prochain mot. On utilisera, dans cette dernière option, l'hypothèse de Markov selon laquelle la probabilité qu'un événement se produit ne dépend que des $o$ derniers événements et non de tous les événements qui se sont produits depuis le début. $o$ représente l'ordre de Markov qui est le nombre d'événements que l'on considère comme influentes pour produire le prochain événement.

Exemple : Pour l'exemple précédent aulieu de prendre en compte toute la sous-séquence "L'audace et la romance semblent être passées à jamais dans le monde", si nous utilisons un bigramme (2-grams), nous ne prendrons que les deux derniers mots de la sous-séquence "le monde". Si par contre nous utilisons un trigramme (3-grams), nous aurons comme sous-séquence résultante "dans le monde". Donc la probabilité que le prochain soit "criminel" peut-être obtenue avec le prochain calcul:

$$
P(prochain\_mot = \text{"criminel"} / \text{bigrammes = "le monde"}) = \frac{\text{nombre d'occurences "le monde criminel"}}{\text{nombre d'occurences "le monde"}}
$$

Effectuons un autre exemple avec la librairie `spacy` qui contient toutes les fonctions dont nous aurons utiliser pour calculer les n-grams, tokéniser les documents et calculer les probabilités/distributions en vue de la prédiction du prochain mot.

In [3]:
import spacy

Chargeons le modèle francais.

In [7]:
nlp = spacy.load("fr_core_news_md")

Utilisons une fonction de tokénisation que nous avions créé pour les tweets. Nous allons fournir le texte que nous avions utiliser au début.

In [11]:
from nlp_project.processing.utils import tokenization

In [14]:
tokens, pos_tags = tokenization(nlp, [texte], rm_stopwords=False, lemmatize=False)

La méthode retourne une liste de tokens pour chaque document et nous n'avons fourni qu'un seul document donc nous obtenons une seule liste que nous allons récupérer.

In [16]:
tokens = tokens[0]

Récupérons à présent les bigrammes.

In [19]:
bigrams = get_n_grams(tokens)

In [20]:
bigrams

['les papiers',
 'papiers sont',
 'sont audace',
 'audace et',
 'et la',
 'la romance',
 'romance semblent',
 'semblent jamais',
 'jamais dans',
 'dans le',
 'le monde',
 'monde criminel',
 'criminel pouvez',
 'pouvez alors',
 'alors me',
 'me demander',
 'demander si',
 'si je',
 'je suis',
 'suis examiner',
 'examiner un',
 'un nouveau',
 'nouveau aussi',
 'aussi insignifiant']